In [1]:
import pandas as pd
import re
import string

from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
from nltk import download
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
QA = pd.read_csv('QA_init.csv', index_col=0)
QA

,question,content,category
0,"Я сменил автомобить, на учет еще не поставил, ...",Для внесения данных по личному автомобилю обра...,автомобиль
1,Не отображается автомобиль в личном кабинете.,Для внесения данных по личному автомобилю обра...,автомобиль
2,добавить автомобиль,Для внесения данных по личному автомобилю обра...,автомобиль
3,хочу внести данные об автомобиле,Для внесения данных по личному автомобилю обра...,автомобиль
4,Как внести данные об автомобиле?,Для внесения данных по личному автомобилю обра...,автомобиль
...,...,...,...
1702,Ошибка 500,Данная ошибка возможна при переходе на портал\...,обучение
1703,при входе на портал обучения выходит ошибка 505,Данная ошибка возможна при переходе на портал\...,обучение
1704,привязать другой номер к карте лояльности,Для изменения привязанного номера к карте лоял...,ЛК
1705,Не открывается итоговый тест на директора мага...,"Создайте, пожалуйста, обращение в ИТ поддержк...",поддержка


In [3]:
download('stopwords')

morph = MorphAnalyzer()

stop_words = set(stopwords.words('russian'))

def preprocess_text(text):
    text = text.lower()  # Приведение к нижнему регистру
    text = re.sub(f"[{string.punctuation}]", "", text)  # Удаление пунктуации
    text = " ".join(morph.parse(word)[0].normal_form for word in text.split() if word not in stop_words)  # Лемматизация и удаление стоп-слов
    return text

QA['question_processed'] = QA['question'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
min_count = 2
category_counts = QA['category'].value_counts()
categories_with_one_record = category_counts[category_counts == 1].index
valid_categories = category_counts[category_counts >= min_count].index

single_record_df = QA[QA['category'].isin(categories_with_one_record)]
filtered_df = QA[QA['category'].isin(valid_categories)]

print(f'Категории после фильтрации: {filtered_df["category"].value_counts()}')

train_df, test_df = train_test_split(filtered_df, test_size=0.2, stratify=filtered_df['category'], random_state=42)
train_df = pd.concat([train_df, single_record_df], axis=0)

print(f'Размер обучающей выборки: {len(train_df)}')
print(f'Размер тестовой выборки: {len(test_df)}')

# Проверка распределения категорий в обучающей и тестовой выборках
print(f'Категории в обучающей выборке: {train_df["category"].value_counts()}')
print(f'Категории в тестовой выборке: {test_df["category"].value_counts()}')

Категории после фильтрации: ЛК                     481
поддержка              276
табель                 168
отпуск                 119
удаленная работа        98
увольнение              60
моя карьера             56
БиР                     50
отгул                   48
прием на работу         48
заявки                  47
зарплата                45
ЭЦП                     33
график работы           28
документооборот         22
больничный              17
налоговый вычет         12
уход за больным          9
справка                  9
автомобиль               8
материальная помощь      8
доверенность             7
обучение                 7
командировка             5
оператор                 5
дмс                      3
перевод                  2
Name: category, dtype: int64
Размер обучающей выборки: 1341
Размер тестовой выборки: 335
Категории в обучающей выборке: ЛК                     385
поддержка              221
табель                 134
отпуск                  95
удаленная работ

In [9]:
# Преобразование текста в Bag of Words
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_df['question_processed'])
X_test = vectorizer.transform(test_df['question_processed'])

y_train = train_df['content']
y_test = test_df['content']

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [10]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.5880597014925373
Precision: 0.49636944972484254
Recall: 0.5880597014925373
F1 Score: 0.5211195070397183


C:\Users\nikit\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\nikit\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
# Преобразование текста в TF-IDF
vectorizer_tf = TfidfVectorizer()
X_train_tf = vectorizer_tf.fit_transform(train_df['question_processed'])
X_test_tf = vectorizer_tf.transform(test_df['question_processed'])

model2 = LogisticRegression()
model2.fit(X_train_tf, y_train)

y_pred_2 = model2.predict(X_test_tf)

In [12]:
accuracy = accuracy_score(y_test, y_pred_2)
precision = precision_score(y_test, y_pred_2, average='weighted')
recall = recall_score(y_test, y_pred_2, average='weighted')
f1 = f1_score(y_test, y_pred_2, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.48059701492537316
Precision: 0.40016765312669517
Recall: 0.48059701492537316
F1 Score: 0.397248736085933


C:\Users\nikit\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
